[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


In [1]:
import requests
import base64
from copy import deepcopy
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob
from bs4 import BeautifulSoup as bsoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
import re
import ast
import emoji

In [2]:
#Makarands Keys (remember to remove)

client_key = "uDxiPfIPIcFjGD0fmxj7c2XAk"
client_secret = "Bcr55JD2dNhGFftvtWwn19x78pRPLNP5rCBmGw0yVeW5Uy3hpc" 

In [3]:
key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [4]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [5]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [6]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAEskIQEAAAAAqkugIbkkPuB3XUUBhHAYkbYX4ik%3DjxVjLVRb5O9OUZ95hy8Z3cDHaTkA4j7VJM0Shx3GNLq2W8SdOA'}

In [7]:
access_token = json_data['access_token']

# Get Tweets
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [8]:
max_tweets=1000 #CHANGED TO 200 FOR TESTING - MUST BE at least 1000 for analysis
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []

In [9]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

keywords = ["Packers","DaBears","Eagles", "Cowboys","Raiders", "ChiefsKingdom"]

for keyword in keywords:
    #parameters popular and extended tweet mode
    parameters = { 'q': keyword,
                        'result_type': 'mixed',
                    'count': tw_limit, 
                     'tweet_mode': 'extended'}

    search_url = base_url+'1.1/search/tweets.json'

    response = requests.get(search_url, headers=search_headers, params=parameters)
    football_tweets = response.json()

    for status in football_tweets['statuses']:
        status["team"] = keyword
    tweets += football_tweets['statuses']
    ids = [tw['id'] for tw in football_tweets['statuses']]
    min_id = min(ids)
    tw_ids = []
    tw_ids+=ids

    for  i in range(max_tweets//tw_limit -1):
        parameters = { 'q': keyword,
                        'result_type': 'mixed',
                    'count': tw_limit,
                    'max_id': min(tw_ids)-1
                     }
        print("searching tweets with id  < {}".format(min_id))
        search_url = base_url+'1.1/search/tweets.json'
        response = requests.get(search_url, headers=search_headers, params=parameters)
        football_tweets = response.json()
        ids = [tw['id'] for tw in football_tweets['statuses']]
        tw_ids+=ids
        print(i, len(football_tweets['statuses']))
        min_id = min(ids)
        for status in football_tweets['statuses']:
            status["team"] = keyword
        tweets += football_tweets['statuses']    

searching tweets with id  < 1329443531694481411
0 100
searching tweets with id  < 1329435997503971329
1 100
searching tweets with id  < 1329427456558829568
2 100
searching tweets with id  < 1329420217181745160
3 100
searching tweets with id  < 1329407920069619713
4 100
searching tweets with id  < 1329398362630746113
5 100
searching tweets with id  < 1329380546359799808
6 100
searching tweets with id  < 1329350284771028999
7 100
searching tweets with id  < 1329329843885010945
8 100
searching tweets with id  < 1329214126946652162
0 100
searching tweets with id  < 1329158885953953794
1 100
searching tweets with id  < 1329109680208097282
2 100
searching tweets with id  < 1329030068752945153
3 100
searching tweets with id  < 1328875802486837252
4 100
searching tweets with id  < 1328829734680190977
5 100
searching tweets with id  < 1328790274148085760
6 100
searching tweets with id  < 1328764987897208834
7 100
searching tweets with id  < 1328742680172572673
8 100
searching tweets with id  < 

As per twitter api documentation we can use

max_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [10]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 6000


# Let's print some tweets

In [11]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [12]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Thu Nov 19 15:16:52 +0000 2020',
  'display_text_range': [0, 202],
  'entities': {'hashtags': [],
               'media': [{'display_url': 'pic.twitter.com/xlo5NGg6iD',
                          'expanded_url': 'https://twitter.com/briebriejoy/status/1329443531694481411/photo/1',
                          'id': 1329443527311450121,
                          'id_str': '1329443527311450121',
                          'indices': [203, 226],
                          'media_url': 'http://pbs.twimg.com/media/EnMh2CpXYAkQrUs.jpg',
                          'media_url_https': 'https://pbs.twimg.com/media/EnMh2CpXYAkQrUs.jpg',
                          'sizes': {'large': {'h': 444,
                                              'resize': 'fit',
                                              'w': 1125},
                                    'medium': {'h': 444,
                                               'resize': 'fit',
           

           'protected': False,
           'screen_name': 'NFLNEWS341',
           'statuses_count': 780,
           'time_zone': None,
           'translator_type': 'none',
           'url': None,
           'utc_offset': None,
           'verified': False}}]


Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [13]:
for status in tweets[0:max_tweet_to_print]:
    print(status['full_text'] + '\n')

Thinking about how many Iowa meat packers voted for Bernie who would’ve given them ownership stake in their business, greater control over their conditions, and Medicare for All. https://t.co/wef0Do6kyu https://t.co/xlo5NGg6iD

Packers' WR Davante Adams did not practice today due to an ankle injury.

Says Darius Leonard, on Packers QB Aaron Rodgers calling San Fran's Fred Warner "the best linebacker in the game" a few weeks ago:

"I've been thinking about that ever since he said that. I hope I can change his mind."

Packers at Colts on Sunday 🍿

@ChicagoBears Try Out Former @packers @Browns And @Raiders QB DeShone Kizer #Browns #chicagobears #packers #raiders #nfl https://t.co/KQZgsqy0nw



# Pandas DataFrame for storing tweet feed


# Printing keys in tweets

In [14]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang', 'team'])
26


In [15]:
import pandas as pd

# Generators for building DataFrames

In [16]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= {}
        
        #Write your code here

        modified_tweet = deepcopy(tweet)
        try:
            modified_tweet["user_id"] = modified_tweet["user"]["id"]
            modified_tweet.pop("user", None)
        except KeyError:
            pass
        try:
            modified_tweet["retweeted_status_id"] = modified_tweet["retweeted_status"]["id"] 
            modified_tweet.pop("retweeted_status", None)
        except KeyError:
            pass
        yield modified_tweet       
            
            

In [17]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'created_at': 'Thu Nov 19 15:16:52 +0000 2020',
  'id': 1329443531694481411,
  'id_str': '1329443531694481411',
  'full_text': 'Thinking about how many Iowa meat packers voted for Bernie who would’ve given them ownership stake in their business, greater control over their conditions, and Medicare for All. https://t.co/wef0Do6kyu https://t.co/xlo5NGg6iD',
  'truncated': False,
  'display_text_range': [0, 202],
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [],
   'urls': [{'url': 'https://t.co/wef0Do6kyu',
     'expanded_url': 'https://twitter.com/dmregister/status/1329431739475832835',
     'display_url': 'twitter.com/dmregister/sta…',
     'indices': [179, 202]}],
   'media': [{'id': 1329443527311450121,
     'id_str': '1329443527311450121',
     'indices': [203, 226],
     'media_url': 'http://pbs.twimg.com/media/EnMh2CpXYAkQrUs.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/EnMh2CpXYAkQrUs.jpg',
     'url': 'https://t.co/xlo5NGg6iD',
     'display

In [18]:
football_feed_df = pd.DataFrame(tweet_generator(tweets))

# Text Cleanup

Here, we're going to take to start cleaning the text in the tweets.

In [19]:
print("Rows with either null text of teams: {}".format(football_feed_df[["full_text","team"]].isnull().any(axis = 1).sum())) 

Rows with either null text of teams: 5400


In [20]:
def NoneToNan(elem):
    if elem == "None" or elem == None:
        return np.nan
    else:
        return elem

football_feed_df = football_feed_df.applymap(NoneToNan)

In [21]:
football_feed_df[["full_text","team", "text"]]

,full_text,team,text
0,Thinking about how many Iowa meat packers vote...,Packers,NaN
1,Packers' WR Davante Adams did not practice tod...,Packers,NaN
2,"Says Darius Leonard, on Packers QB Aaron Rodge...",Packers,NaN
3,@ChicagoBears Try Out Former @packers @Browns ...,Packers,NaN
4,Packers QB Jordan Love moved up to the Superst...,Packers,NaN
...,...,...,...
5995,NaN,ChiefsKingdom,RT @quiquegaray: 🔥😎 ¡Llegaron mis Power Rankin...
5996,NaN,ChiefsKingdom,RT @PFF_Chiefs: New #Chiefs Signing\nP Johnny ...
5997,NaN,ChiefsKingdom,RT @RaiderCody: Hey #ChiefsKingdom........... ...
5998,NaN,ChiefsKingdom,RT @ArrowheadLive: I think he looks better in ...


In [22]:
football_feed_df.full_text.fillna(football_feed_df.text, inplace=True)

football_feed_df[["full_text","team", "text"]]

,full_text,team,text
0,Thinking about how many Iowa meat packers vote...,Packers,NaN
1,Packers' WR Davante Adams did not practice tod...,Packers,NaN
2,"Says Darius Leonard, on Packers QB Aaron Rodge...",Packers,NaN
3,@ChicagoBears Try Out Former @packers @Browns ...,Packers,NaN
4,Packers QB Jordan Love moved up to the Superst...,Packers,NaN
...,...,...,...
5995,RT @quiquegaray: 🔥😎 ¡Llegaron mis Power Rankin...,ChiefsKingdom,RT @quiquegaray: 🔥😎 ¡Llegaron mis Power Rankin...
5996,RT @PFF_Chiefs: New #Chiefs Signing\nP Johnny ...,ChiefsKingdom,RT @PFF_Chiefs: New #Chiefs Signing\nP Johnny ...
5997,RT @RaiderCody: Hey #ChiefsKingdom........... ...,ChiefsKingdom,RT @RaiderCody: Hey #ChiefsKingdom........... ...
5998,RT @ArrowheadLive: I think he looks better in ...,ChiefsKingdom,RT @ArrowheadLive: I think he looks better in ...


In [23]:
hashtags = []
user_mentions = []

for row in football_feed_df.itertuples():
    row = row.entities
    try:
        hashtags_in_row = row["hashtags"]
        hashtags.append(hashtags_in_row)
    except:
        hashtags.append('')
    try:
        user_mentions_in_row = row["user_mentions"][0]["name"]
        user_mentions.append(user_mentions_in_row)
    except:
        user_mentions.append('')



In [24]:
football_feed_df["hashtags"] = hashtags
football_feed_df["user_mentions"] = user_mentions
football_feed_df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,metadata,source,...,favorited,retweeted,possibly_sensitive,lang,team,user_id,retweeted_status_id,text,hashtags,user_mentions
0,Thu Nov 19 15:16:52 +0000 2020,1329443531694481411,1329443531694481411,Thinking about how many Iowa meat packers vote...,False,"[0, 202]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1329443527311450121, 'id_str...","{'result_type': 'popular', 'iso_language_code'...","<a href=""http://twitter.com/download/iphone"" r...",...,False,False,False,en,Packers,1332630685,NaN,NaN,[],
1,Thu Nov 19 21:19:15 +0000 2020,1329534730115702787,1329534730115702787,Packers' WR Davante Adams did not practice tod...,False,"[0, 72]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'result_type': 'popular', 'iso_language_code'...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,False,False,NaN,en,Packers,51263592,NaN,NaN,[],
2,Thu Nov 19 17:22:17 +0000 2020,1329475092603146241,1329475092603146241,"Says Darius Leonard, on Packers QB Aaron Rodge...",False,"[0, 248]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'result_type': 'popular', 'iso_language_code'...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,False,False,NaN,en,Packers,37568810,NaN,NaN,[],
3,Fri Nov 20 20:49:43 +0000 2020,1329889684009463809,1329889684009463809,@ChicagoBears Try Out Former @packers @Browns ...,False,"[0, 120]","{'hashtags': [{'text': 'Browns', 'indices': [7...","{'media': [{'id': 1329889658545836036, 'id_str...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,False,False,True,en,Packers,833663232594288640,NaN,NaN,"[{'text': 'Browns', 'indices': [76, 83]}, {'te...",Chicago Bears
4,Fri Nov 20 20:49:33 +0000 2020,1329889641034637313,1329889641034637313,Packers QB Jordan Love moved up to the Superst...,False,"[0, 206]","{'hashtags': [{'text': 'Packers', 'indices': [...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,False,False,NaN,en,Packers,4845762543,NaN,NaN,"[{'text': 'Packers', 'indices': [188, 196]}, {...",ComboLG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Wed Nov 18 04:30:43 +0000 2020,1328918534752141312,1328918534752141312,RT @quiquegaray: 🔥😎 ¡Llegaron mis Power Rankin...,False,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'es', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,False,False,NaN,es,ChiefsKingdom,32167964,1.328915e+18,RT @quiquegaray: 🔥😎 ¡Llegaron mis Power Rankin...,[],Enrique Garay
5996,Wed Nov 18 04:30:40 +0000 2020,1328918524195188736,1328918524195188736,RT @PFF_Chiefs: New #Chiefs Signing\nP Johnny ...,False,NaN,"{'hashtags': [{'text': 'Chiefs', 'indices': [2...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,False,False,NaN,en,ChiefsKingdom,853743000,1.328897e+18,RT @PFF_Chiefs: New #Chiefs Signing\nP Johnny ...,"[{'text': 'Chiefs', 'indices': [20, 27]}, {'te...",PFF KC Chiefs
5997,Wed Nov 18 04:28:52 +0000 2020,1328918069498941440,1328918069498941440,RT @RaiderCody: Hey #ChiefsKingdom........... ...,False,NaN,"{'hashtags': [{'text': 'ChiefsKingdom', 'indic...","{'media': [{'id': 1328722880662700041, 'id_str...","{'iso_language_code': 'und', 'result_type': 'r...","<a href=""http://twitter.com/download/iphone"" r...",...,False,False,False,und,ChiefsKingdom,890405652794933248,1.328723e+18,RT @RaiderCody: Hey #ChiefsKingdom........... ...,"[{'text': 'ChiefsKingdom', 'indices': [20, 34]}]",Raider Cody
5998,Wed Nov 18 04:28:37 +0000 2020,1328918005888258049,1328918005888258049,RT @ArrowheadLive: I think he looks better in ...,False,NaN,"{'hashtags': [{'text': 'Chiefs', 'indices': [9...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twi

In [25]:
# write code here

def clean_tweet(text):  

    url = re.compile('https://.+')
    rt = re.compile("RT")
    user = re.compile('@\S+\s')
 
    #'(RT|@.*\s|https://.+)'

    text = url.sub("", text)
    text = rt.sub("", text)
    text = user.sub("", text)

    return text

clean_tweet("RT .@SteveKingIA says he's unsure about the science of climate change. But you don’t need to be a scientist to take ac… https://t.co/J1CcrQZkB2")

football_feed_df["full_text"] = football_feed_df["full_text"].apply(clean_tweet)

In [26]:

def emoji_retriever(text):
      return [(char, emoji.demojize(char)) for char in text if char in emoji.UNICODE_EMOJI]

football_feed_df["emoji"] = football_feed_df["full_text"].apply(emoji_retriever)

In [27]:
contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
(r'haven\'t', 'have not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)\'re', '\g<1> are')]

class contraction_replacer(object):
    def __init__(self, contraction_patterns):
        self._contraction_regexes = [(re.compile(p), replaced_text) for p, replaced_text in contraction_patterns]

    def do_contraction_normalization(self, text):
        for contraction_regex, replaced_text in self._contraction_regexes:
            text = contraction_regex.sub(replaced_text, text)
        return text

def preprocess_text(text):
    
    contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
    (r'haven\'t', 'have not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)\'re', '\g<1> are')]

    replacer = contraction_replacer(contraction_patterns)
    text = replacer.do_contraction_normalization(text)
    text = nltk.tokenize.word_tokenize(text)

    stop_words = set(stopwords.words('english')) 
    
    # lemmatization
    wnlemma = WordNetLemmatizer()
    lemma_text = []
    for word in text:
        if word in stop_words:
            continue
        lemma_text.append(wnlemma.lemmatize(word))
    lemma_text = " ".join(lemma_text)

    return lemma_text

football_feed_df["processed_text"] = football_feed_df["full_text"].apply(preprocess_text)

In [28]:
#Pickle dataframe
import pickle
football_feed_df.to_pickle('football_feed_df.pkl')

In [29]:
def sentiment_analysis(text):
    text = " ".join(text["processed_text"])
    text = TextBlob(text)

    return text.sentiment.polarity

teams_df = football_feed_df[["processed_text", "team"]].groupby("team")
sentiments = teams_df.apply(sentiment_analysis)

# Sentiment Analysis

Below represents the relative polarity of each team from each team on Twitter.

In [30]:
sentiments

team
ChiefsKingdom    0.168767
Cowboys          0.078894
DaBears          0.086424
Eagles           0.082551
Packers          0.153226
Raiders          0.087564
dtype: float64

In [31]:
def tweet_user_generator(tweets):
    for tweet in tweets:
        yield tweet["user"]
                 

In [32]:
user_df = pd.DataFrame(tweet_user_generator(tweets))

In [33]:
user_df.head()

,id,id_str,name,screen_name,location,description,url,entities,protected,followers_count,...,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type
0,1332630685,1332630685,BJG 👑,briebriejoy,"Manhattan, NY",Former National Press Secretary for #BernieSan...,https://t.co/7M04QT5ytf,{'url': {'urls': [{'url': 'https://t.co/7M04QT...,False,328110,...,000000,000000,False,True,False,False,None,None,None,none
1,51263592,51263592,Adam Schefter,AdamSchefter,New York,ESPN Senior NFL Insider. \nInterview & Podcast...,https://t.co/OUz4GsIjV2,{'url': {'urls': [{'url': 'https://t.co/OUz4Gs...,False,8030009,...,95E8EC,16101F,True,True,False,False,None,None,None,none
2,37568810,37568810,Zak Keefer,zkeefer,,"Dad to two awesome little girls, Colts writer ...",None,{'description': {'urls': []}},False,41887,...,E3E2DE,634047,True,False,False,False,None,None,None,none
3,833663232594288640,833663232594288640,NFLNEWS34,NFLNEWS341,,,None,{'description': {'urls': []}},False,15,...,DDEEF6,333333,True,False,True,False,None,None,None,none
4,4845762543,4845762543,PackersCombo🧀🧀🧀,ComboProphet,Germany,Madden 21 @ComboLg member. \n\nTime to get rig...,None,{'description': {'urls': []}},False,26,...,DDEEF6,333333,True,True,True,False,None,None,None,none


In [34]:
user_df.to_pickle('user_df.pkl')